In [ ]:
conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1

In [ ]:
pip install tensorflow==2.11

In [ ]:
conda install -c conda-forge cudatoolkit=11.2

In [1]:
# What version of Python do you have?
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.11.0
Keras Version: 2.11.0

Python 3.9.16 (main, Mar  8 2023, 14:00:05) 
[GCC 11.2.0]
Pandas 1.5.3
Scikit-Learn 1.2.2
GPU is available


In [2]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import cross_val_predict, KFold
import sklearn.preprocessing

import time
import sklearn
import numpy as np
import os
import sys
from joblib import parallel_backend

In [3]:
# define a list of datasets
datasets = ["PHM2022_Multivar", "PHM2022_Univar_PDIN"]
datasets_path = "../datasets"

for dataset in datasets:
    Dataset_name = dataset + "_Dataset"
    Dataset = np.load(datasets_path + "/" + Dataset_name + ".npy")
    print(Dataset.shape)
    

    Labels_name = dataset + "_Labels"
    Labels = np.load(datasets_path + "/"  + Labels_name + ".npy")

(53785, 749, 3)
(53785, 749, 1)


In [ ]:
# change this directory for your machine
root_dir = './'

# define the number of folds
n_folds = 5

# perform cross-validation for each dataset and algorithm combination
for dataset in datasets:
    Dataset_name = dataset + "_Dataset"
    Dataset = np.load(datasets_path + "/" + Dataset_name + ".npy")
    

    Labels_name = dataset + "_Labels"
    Labels = np.load(datasets_path + "/"  + Labels_name + ".npy")
    


    # Create a folder for results
    results_path = root_dir + "Results/" + Dataset_name
    if os.path.exists(results_path):
        pass
    else:
        try:
            os.makedirs(results_path)
        except:
            # in case another machine created the path meanwhile !:(
            pass

        
    t_total = time.time() ##Start timing
    


    print(f"\n The dataset shape is:{Dataset.shape}")
    print(f"\n The number of data samples (N) is:{Dataset.shape[0]}")
    print(f"\n The number of TS length (T) is:{Dataset.shape[1]}")
    print(f"\n The number of TS dimention (M) is:{Dataset.shape[2]}")

    nb_classes = len(np.unique(Labels, axis=0))
    
    # transform the labels from integers to one hot vectors
    enc = sklearn.preprocessing.OneHotEncoder(categories='auto')
    Labels = enc.fit_transform(Labels.reshape(-1, 1)).toarray()


    kf = KFold(n_splits=n_folds, shuffle=True)
    accuracy_scores = []
    f1_scores = []
    confusion_matrices = []
    report_list = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(Dataset)):
        # split the data into training and testing sets
        X_train, X_test = Dataset[train_idx], Dataset[test_idx]
        y_train, y_test = Labels[train_idx], Labels[test_idx]
        
        # save orignal y because later we will use binary
        y_true = np.argmax(y_test, axis=1)
        
        input_shape = X_train.shape[1:]
        
        ## Create Classification module
        from dl4tsc.classifiers import mlp
        classifier = mlp.Classifier_MLP(output_directory=results_path, input_shape=input_shape,
                                        nb_classes=nb_classes, verbose=True)
        
        # fit the algorithm on the training data
        classifier.fit(X_train, y_train, X_test, y_test, y_true)
            
        # calculate the evaluation metrics
        accuracy = accuracy_score(y_test, y_pred)
        print(accuracy)

        f1 = f1_score(y_test, y_pred, average='weighted')
        print(f1)

        confusion = confusion_matrix(y_test, y_pred)
        print(confusion)

        accuracy_scores.append(accuracy)
        f1_scores.append(f1)
        confusion_matrices.append(confusion)

        report = classification_report(y_test, y_pred, zero_division=1)
        report_list.append(report)
        print(report)
        
        print(f" fold {fold+1} is Finished!")
        
        # save the output to a text file
        with open(f'{results_path}/dataset_{dataset_name}_LR_fold_{fold+1}.txt', 'w') as f:
            f.write(f'Accuracy: {accuracy}\n')
            f.write(f'F1 Score: {f1}\n')
            f.write(f'Confusion Matrix:\n{confusion}\n\n')
            f.write(f'Classification report:\n{report}\n\n')
        
    with open(f'{results_path}/dataset_{dataset_name}_LR.txt', 'w') as f:
        f.write("Mean accuracy: {:.3f} (std={:.3f})\n".format(np.mean(accuracy_scores), np.std(accuracy_scores)))
        f.write("Mean F1 score: {:.3f} (std={:.3f})\n".format(np.mean(f1_scores), np.std(f1_scores)))
        f.write("Mean confusion matrix:\n{}\n".format(np.array2string(np.mean(confusion_matrices, axis=0))))

    print(" Finished!")
    print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

    



 The dataset shape is:(53785, 749, 3)

 The number of data samples (N) is:53785

 The number of TS length (T) is:749

 The number of TS dimention (M) is:3
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 749, 3)]          0         
                                                                 
 flatten (Flatten)           (None, 2247)              0         
                                                                 
 dropout (Dropout)           (None, 2247)              0         
                                                                 
 dense (Dense)               (None, 500)               1124000   
                                                                 
 dropout_1 (Dropout)         (None, 500)               0         
                                                                 
 dense_1 (Dense)             (None, 5

2690/2690 [==============================] - 8s 3ms/step - loss: 0.5556 - accuracy: 0.8042 - val_loss: 0.3468 - val_accuracy: 0.8894 - lr: 0.0010
Epoch 41/5000
2690/2690 [==============================] - 7s 3ms/step - loss: 0.5435 - accuracy: 0.8045 - val_loss: 0.3393 - val_accuracy: 0.8897 - lr: 0.0010
Epoch 42/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.5297 - accuracy: 0.8081 - val_loss: 0.3339 - val_accuracy: 0.8937 - lr: 0.0010
Epoch 43/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.5225 - accuracy: 0.8122 - val_loss: 0.3262 - val_accuracy: 0.8948 - lr: 0.0010
Epoch 44/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.5157 - accuracy: 0.8128 - val_loss: 0.3172 - val_accuracy: 0.8970 - lr: 0.0010
Epoch 45/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.5056 - accuracy: 0.8166 - val_loss: 0.3136 - val_accuracy: 0.8951 - lr: 0.0010
Epoch 46/5000
2690/2690 [=============================

2690/2690 [==============================] - 8s 3ms/step - loss: 0.2920 - accuracy: 0.8920 - val_loss: 0.1655 - val_accuracy: 0.9412 - lr: 0.0010
Epoch 92/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.2902 - accuracy: 0.8930 - val_loss: 0.1645 - val_accuracy: 0.9412 - lr: 0.0010
Epoch 93/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.2881 - accuracy: 0.8934 - val_loss: 0.1649 - val_accuracy: 0.9402 - lr: 0.0010
Epoch 94/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.2848 - accuracy: 0.8949 - val_loss: 0.1605 - val_accuracy: 0.9439 - lr: 0.0010
Epoch 95/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.2831 - accuracy: 0.8948 - val_loss: 0.1604 - val_accuracy: 0.9424 - lr: 0.0010
Epoch 96/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.2784 - accuracy: 0.8978 - val_loss: 0.1584 - val_accuracy: 0.9443 - lr: 0.0010
Epoch 97/5000
2690/2690 [=============================

2690/2690 [==============================] - 8s 3ms/step - loss: 0.2067 - accuracy: 0.9250 - val_loss: 0.1125 - val_accuracy: 0.9606 - lr: 0.0010
Epoch 143/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.2063 - accuracy: 0.9250 - val_loss: 0.1103 - val_accuracy: 0.9613 - lr: 0.0010
Epoch 144/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.2056 - accuracy: 0.9261 - val_loss: 0.1102 - val_accuracy: 0.9621 - lr: 0.0010
Epoch 145/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.2021 - accuracy: 0.9265 - val_loss: 0.1121 - val_accuracy: 0.9599 - lr: 0.0010
Epoch 146/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.2007 - accuracy: 0.9273 - val_loss: 0.1105 - val_accuracy: 0.9610 - lr: 0.0010
Epoch 147/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.1990 - accuracy: 0.9281 - val_loss: 0.1093 - val_accuracy: 0.9611 - lr: 0.0010
Epoch 148/5000
2690/2690 [=======================

Epoch 193/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.1625 - accuracy: 0.9421 - val_loss: 0.0826 - val_accuracy: 0.9738 - lr: 0.0010
Epoch 194/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.1612 - accuracy: 0.9441 - val_loss: 0.0845 - val_accuracy: 0.9715 - lr: 0.0010
Epoch 195/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.1611 - accuracy: 0.9421 - val_loss: 0.0816 - val_accuracy: 0.9737 - lr: 0.0010
Epoch 196/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.1601 - accuracy: 0.9429 - val_loss: 0.0822 - val_accuracy: 0.9729 - lr: 0.0010
Epoch 197/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.1590 - accuracy: 0.9454 - val_loss: 0.0811 - val_accuracy: 0.9741 - lr: 0.0010
Epoch 198/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.1574 - accuracy: 0.9444 - val_loss: 0.0805 - val_accuracy: 0.9748 - lr: 0.0010
Epoch 199/5000
2690/2690 [========

...

Epoch 601/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0620 - accuracy: 0.9808 - val_loss: 0.0258 - val_accuracy: 0.9947 - lr: 0.0010
Epoch 602/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0630 - accuracy: 0.9804 - val_loss: 0.0267 - val_accuracy: 0.9943 - lr: 0.0010
Epoch 603/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0653 - accuracy: 0.9790 - val_loss: 0.0265 - val_accuracy: 0.9941 - lr: 0.0010
Epoch 604/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0623 - accuracy: 0.9805 - val_loss: 0.0261 - val_accuracy: 0.9943 - lr: 0.0010
Epoch 605/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0651 - accuracy: 0.9800 - val_loss: 0.0263 - val_accuracy: 0.9942 - lr: 0.0010
Epoch 606/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0640 - accuracy: 0.9803 - val_loss: 0.0263 - val_accuracy: 0.9943 - lr: 0.0010
Epoch 607/5000
2690/2690 [========

Epoch 652/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0583 - accuracy: 0.9818 - val_loss: 0.0248 - val_accuracy: 0.9948 - lr: 0.0010
Epoch 653/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0582 - accuracy: 0.9815 - val_loss: 0.0247 - val_accuracy: 0.9947 - lr: 0.0010
Epoch 654/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0589 - accuracy: 0.9815 - val_loss: 0.0250 - val_accuracy: 0.9948 - lr: 0.0010
Epoch 655/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0601 - accuracy: 0.9808 - val_loss: 0.0250 - val_accuracy: 0.9946 - lr: 0.0010
Epoch 656/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0575 - accuracy: 0.9819 - val_loss: 0.0251 - val_accuracy: 0.9949 - lr: 0.0010
Epoch 657/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0611 - accuracy: 0.9807 - val_loss: 0.0248 - val_accuracy: 0.9950 - lr: 0.0010
Epoch 658/5000
2690/2690 [========

Epoch 703/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0572 - accuracy: 0.9815 - val_loss: 0.0236 - val_accuracy: 0.9950 - lr: 0.0010
Epoch 704/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0547 - accuracy: 0.9832 - val_loss: 0.0238 - val_accuracy: 0.9950 - lr: 0.0010
Epoch 705/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0581 - accuracy: 0.9819 - val_loss: 0.0237 - val_accuracy: 0.9950 - lr: 0.0010
Epoch 706/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0545 - accuracy: 0.9835 - val_loss: 0.0237 - val_accuracy: 0.9952 - lr: 0.0010
Epoch 707/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0557 - accuracy: 0.9828 - val_loss: 0.0237 - val_accuracy: 0.9951 - lr: 0.0010
Epoch 708/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0559 - accuracy: 0.9822 - val_loss: 0.0236 - val_accuracy: 0.9951 - lr: 0.0010
Epoch 709/5000
2690/2690 [========

Epoch 754/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0522 - accuracy: 0.9844 - val_loss: 0.0232 - val_accuracy: 0.9951 - lr: 0.0010
Epoch 755/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0527 - accuracy: 0.9840 - val_loss: 0.0229 - val_accuracy: 0.9952 - lr: 0.0010
Epoch 756/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0509 - accuracy: 0.9841 - val_loss: 0.0230 - val_accuracy: 0.9951 - lr: 0.0010
Epoch 757/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0541 - accuracy: 0.9833 - val_loss: 0.0228 - val_accuracy: 0.9953 - lr: 0.0010
Epoch 758/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0545 - accuracy: 0.9833 - val_loss: 0.0229 - val_accuracy: 0.9952 - lr: 0.0010
Epoch 759/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0526 - accuracy: 0.9825 - val_loss: 0.0230 - val_accuracy: 0.9953 - lr: 0.0010
Epoch 760/5000
2690/2690 [========

Epoch 805/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0510 - accuracy: 0.9845 - val_loss: 0.0224 - val_accuracy: 0.9951 - lr: 0.0010
Epoch 806/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0505 - accuracy: 0.9836 - val_loss: 0.0222 - val_accuracy: 0.9950 - lr: 0.0010
Epoch 807/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0523 - accuracy: 0.9838 - val_loss: 0.0221 - val_accuracy: 0.9953 - lr: 0.0010
Epoch 808/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0534 - accuracy: 0.9833 - val_loss: 0.0221 - val_accuracy: 0.9953 - lr: 0.0010
Epoch 809/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0504 - accuracy: 0.9844 - val_loss: 0.0218 - val_accuracy: 0.9953 - lr: 0.0010
Epoch 810/5000
2690/2690 [==============================] - 8s 3ms/step - loss: 0.0491 - accuracy: 0.9848 - val_loss: 0.0222 - val_accuracy: 0.9952 - lr: 0.0010
Epoch 811/5000
2690/2690 [========

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import pandas as pd

def calculate_metrics(y_true, y_pred, duration, y_true_val=None, y_pred_val=None):
    res = pd.DataFrame(data=np.zeros((1, 4), dtype=np.float), index=[0],
                       columns=['precision', 'accuracy', 'recall', 'duration'])
    res['precision'] = precision_score(y_true, y_pred, average='macro')
    res['accuracy'] = accuracy_score(y_true, y_pred)

    if not y_true_val is None:
        # this is useful when transfer learning is used with cross validation
        res['accuracy_val'] = accuracy_score(y_true_val, y_pred_val)

    res['recall'] = recall_score(y_true, y_pred, average='macro')
    res['duration'] = duration
    return res

In [ ]:
df_metrics = calculate_metrics(y_true, y_pred, 0.0)

In [ ]:
classifier.predict(X_test, y_true,X_train,y_train,y_test,return_df_metrics = True)